---
title: "从零开始自己手写语言模型 (二): 文本数据的处理方法"
date: 2025-07-20T09:05:03-05:00
author: "郝鸿涛"
slug: llm
draft: false
toc: false
tags: llm
---

计算机无法直接处理文本，我们需要把文本信息转换为计算机可以理解的数字。怎么做呢？你可以这样想：我们是否可以给每个字在多维空间中找到一个坐标。我们的目标是，相近的词，坐标要接近。这个方法叫做 Word Emebedding。

如果把这个多维空间中的许多坐标投影到二维，我们希望看到的结果是：

![Word Embedding](img/word_embedding.png)

当然，一个词可以有位置，那我们也可以给每个句子、段落、文章找到位置。这些在 retrieval-augmented generation (RAG) 中比较常用，比如每个电影的介绍，我们确定一个位置，然后用户搜索后，确定搜索文本的位置，然后找到和这个搜索文本位置最接近的电影介绍，最后喂给 LLM，输出结果。具体实现可以看我的[这篇博客](/cn/2025/03/16/bert/)。

下面，我们用一篇文章来展示处理文本的主要步骤。

In [27]:
import urllib.request

# 下载文本信息
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "data/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

# 读取
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [28]:
print("Total number of character:", len(raw_text))

Total number of character: 20479


In [29]:
# 打印前 199 个字符
print(raw_text[:199])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married 


## 文本到 ID 相互转换

之前提到过，计算机只能读取数字。我们需要给每一个 token 安排一个 ID。比如，我们说 I 对应 0, always 对应 1 等等。你可以把这个想象成一个字典或者电台密码本。但有个问题是，小的文本我们可以给每个词一个 id，但是在大语言模型训练中，肯定会出现新的词，以及很奇怪的词，那怎么办？比如，一篇文章里突然出现了 SomeStrangeWordYouHaveNeverSeenBefore 这个词，怎么办？

有一个办法是所有字典里没有的词，统一用 `<|unk|>` 来代替，这里提一句，还有一个非常重要的 id 是 `<|endoftext|>`，用来指示一段文本的结束。这可以在训练时告诉 LLM，下面内容来自不同的来源。

用 `<|unk|>` 的弊端至少有两个：1. 错过一些有意义的陌生词，比如我上面举的那个例子。也就是 LLM 训练时，我们没办法正确给它位置，因为我们看不到它的内容。2. 从 ID 转换到文本这个过程会失败。比如，我们知道 1 对应 always，但是我们肯定无法根据 ID 转换回 SomeStrangeWordYouHaveNeverSeenBefore。

一个更有效的方法是 byte pair encoding (BPE)，也是 GPT 训练时用的方法。我们会用到 OpenAI 开发的包 [tiktoken](https://github.com/openai/tiktoken)。

在 Terminal 输入：

```sh
pip install tiktoken
```

然后正常使用：

In [30]:
import tiktoken
# 初始化
tokenizer = tiktoken.get_encoding('gpt2')

# 测试从文本到 token ID
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces "
    "of someunknownPlace and SomeStrangeWordYouHaveNeverSeenBefore."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 290, 2773, 38114, 26449, 1639, 11980, 12295, 4653, 268, 8421, 13]


In [31]:
# 检测从 ID 回到文本
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace and SomeStrangeWordYouHaveNeverSeenBefore.


我们看到转换很顺利。BEP 的机制如下：

![byte pair encoding](img/byte_pair_encoding.png)

## 准备训练数据

还记得我们之前提到过的 LLM 训练方法吗？根据之前的文本，预测下一个词：

![LLM Prediction](img/llm_predict.png)

那训练数据是不是要这样子：

In [32]:
enc_text = tokenizer.encode(raw_text)
enc_sample = enc_text[50:] # 此步非必须
context_size = 4 

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "----->", desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


我们的最终目标是得到 inputs tensor 和 target tensor:

![Data Loader](img/data_loader.png)

这里有两个问题，或者说变量。第一个是每个 vector 有多大？我们用 `max_length` 表示。这里显示是 4。但这只是为了显示，在真正的 LLM 训练中，`max_length >= 256`。

另一个变量是 `stride`: 一句话结束之后，我们下一句的开始在哪里？

![Stride illustration](img/stride.png)

这里面再加一个变量：`batch size`，也就是训练数据的长度。在图 2.13 中，如果省略号代表的是最后一组数据，那么 `batch size = 4`，因为有四组数据。

In [34]:
import torch 
from torch.utils.data import Dataset, DataLoader 

class GPTDatasetV1(Dataset):
    def __init__(self, txt:str, tokenizer, max_length:int, stride:int) -> None:
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [35]:
GPT_Dataset = GPTDatasetV1(txt=raw_text, tokenizer=tokenizer, max_length=4, stride=4)

# input
GPT_Dataset.input_ids[:5]

[tensor([  40,  367, 2885, 1464]),
 tensor([1807, 3619,  402,  271]),
 tensor([10899,  2138,   257,  7026]),
 tensor([15632,   438,  2016,   257]),
 tensor([ 922, 5891, 1576,  438])]

In [36]:
# target
GPT_Dataset.target_ids[:5]

[tensor([ 367, 2885, 1464, 1807]),
 tensor([ 3619,   402,   271, 10899]),
 tensor([ 2138,   257,  7026, 15632]),
 tensor([ 438, 2016,  257,  922]),
 tensor([5891, 1576,  438,  568])]

我们继续

In [37]:
def create_dataloader_v1(
        txt, batch_size = 4, max_length = 256, stride = 128, shuffle=True,
        drop_last=True, num_workers = 0
):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, 
        batch_size = batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [38]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [39]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


我们试一下不一样的设置：

In [40]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs: \n", inputs)
print("\nTargets:\n", targets)

Inputs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 制作 token embeddings

准备大语言模型训练数据的最后一步是把 token id 转化成 embedding vectors。也就是说，给每一个 token id 代表的文本找到其在 embedding 空间中的位置。

![Token ID to Embedding vectors](img/id2embedding.png)

我们举个例子。假设全世界总共有 6 个字，每个字在三维的 embedding 空间中有坐标：

In [41]:
torch.manual_seed(123)
vocab_size = 6
output_dim = 3
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [42]:
# 假设我们有四个 token ids
input_ids = torch.tensor([2,3,5,1])

# 打印出这四个 token 的 embedding 空间位置
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


但这里有一个问题，这种在 embedding layer 中查找词然后确定 embedding vector 的做法有一个缺陷：它无法告诉我们位置信息。比如，这样一组词：fox jumps over fox。两个 fox 的 embedding vector 是相同的：

![Same embedding vector](img/same_embedding_vector.png)

有两种解决办法。第一种办法是每一个位置有一个相对应的 positional embedding。然后每一个词的 embedding 是 token embedding + positional embedding。这也是 GPT 模型所采用的方法。第二种方法是让模型学习每一个不同位置的词的相对距离，这里我们不展开，大家知道有这种方法即可。

![Absolute positional embeddings](img/absolute_positional_embeddings.png)

好，我们现在用更加现实的方法来举例子。

我们就用刚才 `dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)` 的结果。我们来看一下 `inputs` 的维度：

In [43]:
print("\nInputs shape: \n", inputs.shape)


Inputs shape: 
 torch.Size([8, 4])


我们刚才举的 embedding 例子，用到

```py
vocab_size = 6
output_dim = 3
```

但是在现实中这不现实。比如，GPT-3 的 `output_dim` 高达 12,288。更现实的情况是：

In [44]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

那现在想一下 `token_embedding_layer(inputs)` 会是什么样的维度？

In [45]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


接下来我们制作 position embeddings:

In [48]:
max_length = 4
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


然后我们加工 input embeddings:

![Input embeddings pipeline](img/input_embeddings.png)

In [49]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
